In [1]:
!pip install aguaclara

     |████████████████████████████████| 92kB 3.5MB/s 
     |████████████████████████████████| 163kB 8.4MB/s 
  Created wheel for aguaclara: filename=aguaclara-0.1.10-cp36-none-any.whl size=95255 sha256=9c3c1cc09f4197642b33482b0af43608f7459cd078f51f0bd26ff275f7828b52
  Stored in directory: /root/.cache/pip/wheels/b2/cb/bf/f2bc71c3e00bf3b96a154fe1f01db912d774b56741f7876555
  Created wheel for pint: filename=Pint-0.8.1-py2.py3-none-any.whl size=134155 sha256=f26b2ca9d762a0e2bed1a4e25ba08ca8d0eea64b48131e4447adacc52b7dc26b
  Stored in directory: /root/.cache/pip/wheels/b4/56/36/7c4d6a4e3660e233b95e3b0482551915cb184abca0f93ee1dd
Successfully built aguaclara pint


In [0]:

import aguaclara as ac
from aguaclara.core.units import unit_registry as u
import matplotlib.pyplot as plt
import numpy as np



# Flocculator design

Here I assume we put the entrance tank in the first channel

In [35]:
Train_q=75*u.L/u.s
Floc = ac.Flocculator(q=Train_q,temp=10*u.degC,ent_l=0*u.m, chan_n_parity='odd', l_max=6.3*u.m,)
print('Numero de canales:',Floc.chan_n)
print('Ancho del canal:',Floc.chan_w.to(u.inch))
print('Espacio entre pantallas:',Floc.baffle_s)
print('Tamaño del tubo de drenaje:',Floc.drain_pipe.size)
print('Perdida de carga por el floculador:',Floc.hl)
print('Gt',Floc.gt)
print('Tiempo de retencion',Floc.retention_time.to(u.min))
print('Gradiente de velocidad:',Floc.vel_grad_avg)


Numero de canales: 3
Ancho del canal: 35.83 inch
Espacio entre pantallas: 34.33 centimeter
Tamaño del tubo de drenaje: 4 inch
Perdida de carga por el floculador: 40 centimeter
Gt 37000
Tiempo de retencion 7.561 minute
Gradiente de velocidad: 81.56 / second


#Entrance Tank
No one uses plate settlers in grit chambers. It isn't clear why. The plate settlers enable us to make a more compact system.

Kawamura suggests capture velocites of 10-25 m/hr (3-7 mm/s). We had been using 8 mm/s. I suggest we use the conservative design of 3 mm/s for future plants.
Should the entrance tank have an inlet manifold with diffusers and a jet reverser? It might make sense to make this entrance tank as deep as the flocculator channel.

Kamura suggests a mean velocity in the grit chamber of 3-4.5 m/min (50-75 mm/s)

## Screen
Kamura suggests maximum velocity through the screen net open area of 0.6 m/s. This would correspond to a head loss of of 1.8 cm through the screen.
We need a maximum opening size in the screen that is smaller than the diffuser slot width in the sed tank.

## Turbulence
We want to have the surface of the water in the entrance tank be calm. I'd interpret that to mean less than 1 mm of elevation change from eddies. Eddy velocities are typically 10% of mean velocities. This would set the mean velocity to be less than 1.4 m/s.

## Flow distribution through Entrance Tank plate settlers
Requires low horizontal velocity above and below the plates. The horizontal velocity head has to be small compared with the head loss through the plates.

$$V_{Plate \uparrow} = V_\alpha\sin \alpha$$

$$-\Delta P = 2 \mu \left( \frac{6 V_{Plate \uparrow}}{S \sin^2 \alpha \cos \alpha} \right) \left( \frac{V_{Plate \uparrow}}{V_c} - 1 \right)$$

$$h_{\rm{f}} = \frac{-\Delta P}{\rho g}$$

Combine the previous equations to obtain the head loss as a function of velocity through the plates $V_\alpha$.

$$h_{\rm{f}} = \frac{2 \nu}{g} \left( \frac{6 V_\alpha}{S \sin \alpha \cos \alpha} \right) \left( \frac{V_\alpha\sin \alpha}{V_c} - 1 \right)$$



In [41]:
def HeadLoss_Plate(v,S,angle,v_capture,temp):
  Nu = ac.viscosity_kinematic(temp)
  term1 = 2*Nu/u.gravity
  term2 = 6*v/(S*np.sin(angle)*np.cos(angle))
  term3 = v*np.sin(angle)/v_capture -1
  return (term1*term2*term3).to(u.cm)

print(np.cos(60*u.deg))
#maximum mean velocity given 1 mm eddy head
V_mean = (10*np.sqrt(2*u.gravity*1*u.mm)).to(u.m/u.s)
V_mean

0.5 dimensionless


<Quantity(1.400474919446971, 'meter / second')>

In [46]:

w=42*u.inch
v_capture = 3*u.mm/u.s
temp= 20 * u.degC
ET=ac.EntranceTank(q=Train_q,plate_capture_vel=v_capture,floc_chan_w=w)
my_LFOM=ac.LFOM(q=Train_q,hl=20*u.cm)
print('LFOM ND',ET.lfom_nd)
print('LFOM ND',my_LFOM.pipe_nd)
print('Numero de placas:',ET.plate_n)
print('Largo de las placas:',ET.plate_l)
print('angle of the plates',ET.plate_angle)

v= (q/(w*ET.plate_s*ET.plate_n)).to(u.mm/u.s)
Plate_head_loss = HeadLoss_Plate(v,ET.plate_s,ET.plate_angle,v_capture,temp)
print('plate settler head loss is',Plate_head_loss.to(u.um))
v

LFOM ND 2 inch
LFOM ND 18 inch
Numero de placas: 34 dimensionless
Largo de las placas: 105 centimeter
angle of the plates 50 degree
plate settler head loss is 1.998 micrometer


<Quantity(11.028033260548314, 'millimeter / second')>

Head loss through these plate settlers is negligible and this suggests that a jet reverser might be helpful. And we would need a launder to collect the water from the top of the plates

In [0]:
(4.5*u.m/u.min).to(u.mm/u.s)

In [0]:
((0.6*u.m/u.s)**2/(2*u.gravity)).to(u.cm)